In [67]:
import requests
import sys
import pandas as pd
import time
from datetime import date, timedelta
import socket
import warnings
warnings.filterwarnings("ignore")
import datetime as dt

In [20]:
jan_18=pd.read_csv("Jan_2018.csv")
jan_18.head()

,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,...,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,New York City,2018-01-01T00:00:00,-10.8,-16.6,-15.9,66.03,0.0,0,NaN,0.0,...,1026.4,0.0,16.0,0,NaN,0,NaN,Clear,clear-night,"72505394728,72055399999,KLGA,KJRB,KNYC"
1,New York City,2018-01-01T01:00:00,-11.2,-16.6,-17.3,60.69,0.0,0,NaN,0.0,...,1026.5,0.0,16.0,0,NaN,0,NaN,Clear,clear-night,"72505394728,72055399999,KLGA,KJRB,KNYC"
2,New York City,2018-01-01T02:00:00,-11.7,-16.5,-17.3,63.02,0.0,0,NaN,0.0,...,1026.8,0.0,16.0,0,NaN,0,NaN,Clear,clear-night,"72505394728,72055399999,KLGA,KJRB,KNYC"
3,New York City,2018-01-01T03:00:00,-11.7,-16.5,-17.3,63.17,0.0,0,NaN,0.0,...,1027.2,0.0,16.0,0,NaN,0,NaN,Clear,clear-night,"72505394728,72055399999,KLGA,KJRB,KNYC"
4,New York City,2018-01-01T04:00:00,-11.7,-17.8,-17.4,63.01,0.0,0,NaN,0.0,...,1027.2,0.0,16.0,0,NaN,0,NaN,Clear,clear-night,"72505394728,72055399999,KLGA,KJRB,KNYC"


In [54]:
def cleaning(df):
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['Hour'] = df['datetime'].dt.hour
    df['Date'] = df['datetime'].dt.date
    bins = [0,4,8,12,16,20,24]
    df['HOUR_BIN'] = pd.cut(df['Hour'], bins,labels=["0","4","8","12","16","20"])
    temp=df.groupby(['Date','HOUR_BIN']).mean()
    temp.drop(['severerisk', 'Hour'], axis=1, inplace=True)
    temp.reset_index(inplace=True)
    #temp.index=[0,4,8,12,16,20]
    return temp

In [62]:
df=cleaning(jan_18)
df.columns

C:\Users\omhai\AppData\Local\Temp\ipykernel_26504\330022347.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  temp=df.groupby(['Date','HOUR_BIN']).mean()


Index(['Date', 'HOUR_BIN', 'temp', 'feelslike', 'dew', 'humidity', 'precip',
       'precipprob', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
       'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation',
       'solarenergy', 'uvindex'],
      dtype='object')

In [71]:
year_2018=['Jan_2018.csv','Feb_2018.csv','March_2018.csv','April_2018.csv','May_2018.csv','June_2018.csv','July_2018.csv',
           'August_2018.csv',"September_2018.csv",'October_2018.csv','November_2018.csv','December_2018.csv']
weather_18=pd.DataFrame(columns=df.columns)

for i in year_2018:
    df=pd.read_csv(i)
    df=cleaning(df)
    weather_18=weather_18.append(df, ignore_index = True)

weather_18.head()

,Date,HOUR_BIN,temp,feelslike,dew,humidity,precip,precipprob,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex
0,2018-01-01,0,-11.575,-16.850,-17.325,62.4725,0.0,0.0,0.0,0.0,32.566667,9.350,304.25,1026.925,0.00,16.000,0.00,NaN,0.00
1,2018-01-01,4,-11.975,-16.600,-16.475,69.2450,0.0,0.0,0.0,0.0,35.266667,7.775,292.75,1027.925,0.05,16.000,16.25,0.150,0.00
2,2018-01-01,8,-9.025,-13.350,-15.475,60.3375,0.0,0.0,0.0,0.0,45.100000,8.350,305.50,1028.500,0.25,15.525,330.25,1.175,3.50
3,2018-01-01,12,-5.150,-8.175,-16.250,41.5300,0.0,0.0,0.0,0.0,31.850000,7.075,320.50,1027.150,0.20,16.000,296.75,1.075,3.25
4,2018-01-01,16,-6.350,-9.775,-14.900,50.9925,0.0,0.0,0.0,0.0,42.975000,7.775,305.75,1027.750,0.20,16.000,2.50,0.000,0.00


In [72]:
weather_18.tail()

,Date,HOUR_BIN,temp,feelslike,dew,humidity,precip,precipprob,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex
2185,2018-12-31,4,39.575000,39.575000,33.700,79.280000,0.00000,0.0,0.0,0.0,NaN,0.100000,1.750000,1027.650000,24.575,9.900000,320.75,1.150,0.0
2186,2018-12-31,8,44.000000,40.825000,35.625,72.402500,0.00000,0.0,0.0,0.0,19.000000,6.075000,148.000000,1028.075000,48.625,9.900000,139.50,0.525,1.5
2187,2018-12-31,12,44.475000,40.300000,37.875,77.665000,0.00300,50.0,0.0,0.0,25.900000,7.550000,155.000000,1024.950000,48.800,9.825000,58.00,0.200,0.5
2188,2018-12-31,16,44.050000,40.100000,41.775,91.700000,0.08425,100.0,0.0,0.0,18.766667,7.025000,115.250000,1020.700000,48.775,6.525000,2.00,0.000,0.0
2189,2018-12-31,20,45.366667,40.466667,44.300,96.056667,0.14400,100.0,0.0,0.0,20.700000,9.566667,44.666667,1014.166667,48.500,6.433333,0.00,NaN,0.0
